In [1]:
# %cd ..
import inputs, simuls, analysis
results = analysis.GetResults(verbose=True)

# PenEasy SPC simulation

In [ ]:
pid = 'SPC'
# Initialize PenEasy Input Editor
peneasy_inps = inputs.PenEasy(verbose=True)

# Initialize Simulators (identified by pid)
peneasy_sims = simuls.PenEasy(verbose=True)
peneasy_sims.activate_pid('SPC')

In [ ]:
MATS = {'water' : ['water', 1, 1.00+00]}
SIZE = [10, 10, 10]
STEP = [0.4, 0.4, 0.4]  #cm
nhist = "1e6"

peneasy_inps.edit_voxSize(pid, SIZE, STEP)
peneasy_inps.edit_seed(pid, 12345, 54321)
peneasy_inps.edit_mat(pid, MATS, SIZE, STEP)
peneasy_inps.edit_source_nhist(pid, nhist)

In [ ]:
isotopes = ["N13", "O15", "F18", "Cu64", "Ga68", "Rb82", "I124"] #"C11", 

peneasy_inps.verbose = False
for iso in isotopes:
    print(f"Simulating {iso}")
    peneasy_inps.edit_isotope(pid, iso)
    peneasy_sims.simulate(pid, get_times=True, time_samples=1, output_dir="RESULTS/SPC-Study1-Water/PenEasy_xyz", final_file=f"{iso}")

# vGATE 9.x simulation

In [2]:
pid = '9'
# Initialize GATE Input Editor
gate_inps = inputs.GATE(verbose=True)

# Initialize Simulators (identified by pid)
gate_sims = simuls.GATE(verbose=True)
gate_sims.activate_pid(pid, output_format='dat')

GATE 9 activated


In [3]:
MATS = {'water' : ['Water', 6, 1.00+00]}
SIZE = [10, 10, 10]
STEP = [0.4, 0.4, 0.4]  #cm
nhist = "1e6"

gate_inps.edit_voxSize(pid, SIZE, STEP)
gate_inps.edit_seed(pid, 12345, 54321)
gate_inps.edit_mat(pid, MATS, SIZE, STEP)
gate_inps.edit_source_nhist(pid, nhist)

"gate/mac/actors/ProdStop.mac" voxel dimensions modified to (10, 10, 10) and total size to (4, 4, 4) cm
"gate/phantom/ACTIVITY.h33" and "gate/phantom/MATERIAL.h33" voxel dimensions modified to (10, 10, 10)
"gate/phantom/ACTIVITY.h33" and "gate/phantom/MATERIAL.h33" voxel size modified to (4, 4, 4) mm
"gate/mac/phantoms/analytical/1mat.mac" resized
"gate/mac/phantoms/analytical/2mat.mac" resized
"gate/mac/phantoms/analytical/3mat.mac" resized
"gate/mac/sources/shape/cylinder*.mac" halfz resized
"gate/PR_GATEv7.mac gate/PR_GATEv9.mac" random seed modified to 12345
"mac/phantoms/analytical/1mat.mac" materials modified to Water (case 1)
"gate/PR_GATEv9.mac" number of histories modified to 1e6


In [4]:
isotopes = ["C11", "N13", "O15", "F18", "Cu64", "Ga68", "Rb82", "I124"] #
res_folder = "RESULTS/SPC-Study1-Water/GATE92_xyz"

gate_inps.verbose = False
for iso in isotopes:
    print(f"Simulating {iso}")
    gate_inps.edit_isotope(pid, iso)
    gate_sims.simulate(pid, get_times=True, time_samples=1, output_dir=res_folder, final_file=f"{iso}")


Simulating C11
GATE 9 real time: 508.244 +- 0.000 s
Simulating N13
GATE 9 real time: 350.140 +- 0.000 s
Simulating O15
GATE 9 real time: 202.448 +- 0.000 s
Simulating F18
GATE 9 real time: 338.186 +- 0.000 s
Simulating Cu64
GATE 9 real time: 434.437 +- 0.000 s
Simulating Ga68
GATE 9 real time: 638.798 +- 0.000 s
Simulating Rb82
GATE 9 real time: 406.533 +- 0.000 s
Simulating I124
GATE 9 real time: 134.757 +- 0.000 s


TypeError: Mismatch between array dtype ('<U32') and format specifier ('%.18e %.18e')

In [8]:
import numpy as np
times_file = res_folder + "/" + pid + "_times.txt"
times = np.loadtxt(times_file).astype("str")
times[:,1], times[:,0] = times[:,0], isotopes
np.savetxt(times_file, times, fmt='%s')

# Results analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

histo_bins = 201
histo_step = .01 #cm
isotopes = ["C11", "N13", "O15", "F18", "Cu64", "Ga68", "Rb82", "I124"] 

lowPRisotopes = ["C11", "N13", "Cu64", "F18"]
highPRisotopes = ["O15", "Ga68", "Rb82", "I124"]
lowPRlabels = {iso:[] for iso in lowPRisotopes}
highPRlabels = {iso:[] for iso in highPRisotopes}

results.active_results.clear()
for iso in isotopes:
    of = f"RESULTS/SPC-Study1-Water/GATE92_xyz/{iso}.dat"
    results.load(f"GATEv9.2 {iso}", of, [histo_bins]*3, [histo_step]*3)

    if iso in lowPRlabels:
        lowPRlabels[iso].append(f"GATEv9.2 {iso}")
    elif iso in highPRlabels:
        highPRlabels[iso].append(f"GATEv9.2 {iso}")

# for iso in isotopes:
#     of = f"RESULTS/SPC-Study1-Water/PHITS_xyz/{iso}.dat"
#     results.load(f"PHITS {iso}", of, [histo_bins]*3, [histo_step]*3)

#     of = f"RESULTS/SPC-Study1-Water/PenEasy_xyz/{iso}.dat"
#     results.load(f"PenEasy {iso}", of, [histo_bins]*3, [histo_step]*3)

#     if iso in lowPRlabels:
#         lowPRlabels[iso].append(f"PHITS {iso}")
#         lowPRlabels[iso].append(f"PenEasy {iso}")
#     elif iso in highPRlabels:
#         highPRlabels[iso].append(f"PHITS {iso}")
#         highPRlabels[iso].append(f"PenEasy {iso}")

# for iso in ["C11", "O15", "F18"]:
#     of = f"RESULTS/SPC-Study1-Water/GATE7_xyz/{iso}.dat"
#     results.load(f"GATE7 {iso}", of, [histo_bins]*3, [histo_step]*3)

#     if iso in lowPRlabels:
#         lowPRlabels[iso].append(f"GATE7 {iso}")
#     elif iso in highPRlabels:
#         highPRlabels[iso].append(f"GATE7 {iso}")

lowPRlabels_all = [label for iso in lowPRisotopes for label in lowPRlabels[iso]]
highPRlabels_all = [label for iso in highPRisotopes for label in highPRlabels[iso]]

In [ ]:
results.data_analysis()

In [ ]:
results.plot_aPSFx(sin=True, log_scale=True, labels=highPRlabels["O15"])

In [ ]:
results.plot_aPSFx(sin=True, log_scale=True, lim=6, labels=lowPRlabels_all, legend_size=8)
plt.show()
results.plot_aPSFx(sin=True, log_scale=True, lim=10, labels=highPRlabels_all, legend_size=8)
plt.show()

In [ ]:
results.plot_aPSF3D(sin=True, log_scale=True, labels=lowPRlabels_all, legend_size=8)
plt.show()
results.plot_aPSF3D(sin=True, log_scale=True, labels=highPRlabels_all, legend_size=8)
plt.show()


In [ ]:
results.plot_g3D(log_scale=False, labels=lowPRlabels_all)

In [ ]:
results.plot_G3D(labels=lowPRlabels_all)